<a href="https://colab.research.google.com/github/osipov/edu/blob/master/tf0/Solution_Classifier_kerasDense.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

## Implement a binary classifier with `keras.layers.Dense`

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(42);

## Start by practicing boolean operations on boolean tensors.
* these operations are going to be useful shortly when computing the accuracy metric of your classifier

In [ ]:
a = tf.random.normal([10])
a

## Create a boolean tensor named `b` by converting the `a` tensor such that `b` holds `True` for positive values of `a` and `False` for negative and zero values of `a`
* **hint:** `b` must have the same shape as `a`

In [ ]:
b = a > 0
b

## Create another boolean tensor named `c` with values that are the opposite of `b`
* **hint:** you can use a different logical expression with the values of the `a` tensor or use the `~` operator

In [ ]:
c = ~b
c

## Create 2 tensors that are the logical `and` as well as `or` of `b` and `c`. For each of the tensors, count the number of the `True` values in the tensor
* **hint:** in Python `&` is the logical `and` while `|` is the logical `or`
* **hint:** you'll want to `cast` the boolean tensor before counting

In [ ]:
tf.math.reduce_sum(tf.cast(b & c, tf.uint8)).numpy()

In [ ]:
tf.math.reduce_sum(tf.cast(b | c, tf.uint8)).numpy()

## Create a tensor named `d` by concatenating `b` and `c` into a single row
* try the `tf.concat` method which takes a list of tensors and an axis

In [ ]:
d = tf.concat([b, c], axis = 0)
d

## Reshape the `d` tensor to be 3 dimensional with the shape of `[5, 2, 2]` and save the result to tensor `e`

In [ ]:
e = tf.reshape(d, [5, 2, 2])
e

# Create a tensor `f` that contains the sum of the number of the `True` values along the last (trailing) dimension of the `e` tensor while keeping the original dimensions
* **hint:** check out the `keepdims` parameter

In [ ]:
f = tf.math.reduce_sum(tf.cast(e, tf.uint8), axis = 2, keepdims = True)
f

## Use the `squeeze` method on the tensor with the sum of the `True` values and confirm that its shape changed to 2 dimensions instead of 3

In [ ]:
f.shape, tf.squeeze(f).shape

The `squeeze` method is useful when you need to reduce the dimension of a tensor that has one or more dimensions of length `1`. For example, if you have a tensor with a shape [3, 1, 4], the 2nd dimension can be `squeeze`d to `[3,4]`

## Next, get started on generating data for your spam/not-spam classification problem

In [ ]:
NUM = 50

#X spam data points
Xs = tf.random.normal([NUM, 2], 0, 2) - 3
plt.scatter(Xs[:, 0], Xs[:, 1], color = 'orange');

#X not spam data points
Xns = tf.random.normal([NUM, 2], 0, 3) + 3
plt.scatter(Xns[:, 0], Xns[:, 1], color = 'blue');

plt.xlim([-10, 10])
plt.ylim([-10, 10]);

## Create a tensor array `X` with spam and not spam data values having the shape `[100, 2]`

In [ ]:
X = tf.concat([Xs, Xns], axis = 0)
X.shape

## Create a `y` tensor with positive/negative values for the spam/not spam data in the `X` tensor. Let's have `1` be spam, and `-1` not spam.

In [ ]:
ys = tf.ones([len(Xs)])
yns = -1 * tf.ones([len(Xns)])
y = tf.concat([ys, yns], axis = 0)
y

## Create a model using `keras.layers.Dense`. Disable the bias term in the model.

In [ ]:
model = Dense(input_shape=(2,),
              units = 1,
              activation='linear', 
              use_bias = False)

## Implement the `forward` method for the model. Don't forget to check the shape of your label!

In [ ]:
def forward(X):
  return tf.squeeze(model(X))

y_est = forward(X)
y_est

## Implement the `loss` method to return the mean squared error of your predictions

In [ ]:
def loss(y_est, y):
  return tf.math.reduce_mean(((y_est - y) ** 2))
  
loss(forward(X), y)

## Implement a `metric` method that takes the model predictions and the actual values and returns the accuracy (i.e. percentage correct) for the predictions.

In [ ]:
def metric(y_est, y):
  is_right = (y > 0) & (y_est > 0) | ((y <= 0) & (y_est <= 0))
  num_right = tf.math.reduce_sum(tf.cast(is_right, tf.uint8))
  return num_right / len(is_right)

metric(forward(X), y)

## Implement a `for` loop that does 100 epochs of gradient descent, printing out the MSE and the accurary for each iteration
* **hint:** update the weights using `optimizer.apply_gradients(zip(grad, model.trainable_weights))`

In [ ]:
LEARNING_RATE = 0.03
EPOCHS = 100

In [ ]:
optimizer = Adam(learning_rate = LEARNING_RATE)

for _ in range(EPOCHS):

    with tf.GradientTape() as tape:

      y_est = forward(X)
      mse = loss(y_est, y)

    # backpropagation
    grads = tape.gradient(mse, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    accuracy = metric(y_est, y)
    print(f"MSE: {mse.numpy()} Accuracy: {accuracy.numpy()}")

## Re-render the original scatter plot with spam/not spam data points and add the decision boundary line on the plot.


In [ ]:
x_min, x_max, y_min, y_max, step = -15., 15, -15., 15., 0.5

xx, yy = tf.meshgrid(tf.range(x_min, x_max, step),
                    tf.range(y_min, y_max, step))

X_test = tf.concat([tf.reshape(xx, [tf.size(xx), 1]), 
                tf.reshape(yy, [tf.size(yy), 1])], 
               axis = 1)
y_test = tf.sign(forward(X_test))

Z = -1 * tf.reshape(y_test, xx.shape)

fig = plt.figure()
plt.axis("equal")
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.4);
plt.scatter(X[:NUM, 0], X[:NUM, 1], color = 'orange');
plt.scatter(X[NUM:, 0], X[NUM:, 1]);

## Compare the weights discovered by gradient descent to the weights according to the analytical solution for the problem
* you need the formula $ (X^T X)^{-1}X^Ty $
* **hint:** use `@` for tensor multiplication
* **hint:** use `tf.linalg.inv` for matrix inverse

In [ ]:
model.trainable_weights[0].numpy(),  tf.linalg.inv(tf.transpose(X) @ X) @ tf.transpose(X) @ y[:, None]

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.